# 02 — Análise Exploratória de Dados (EDA)

Projeto: Classificação de Pneumonia em Raio-X  
Liga Acadêmica de Inteligência Artificial — UFPE

## Objetivo

Esta análise exploratória tem caráter **metodológico e decisório**:
cada achado é diretamente vinculado a uma escolha experimental.

| Achado do EDA | Decisão Metodológica |
|---|---|
| Desbalanceamento entre classes | → Hipótese 3: Class Weighting |
| Tamanho moderado do dataset | → Transfer Learning (não treinar do zero) |
| Alta variabilidade de dimensões | → Resize + CenterCrop padronizados |
| Artefatos nas bordas | → CenterCrop (0.9 do resize) |
| Padrões espaciais/texturais | → CNN como arquitetura base |
| Diferenças visuais sutis em alguns casos | → Testar DenseNet121 (H1) |
| Distribuição ampla de intensidades | → Normalização ImageNet |


## 1. Setup

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from PIL import Image
from collections import Counter

NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

figures_dir = os.path.join(PROJECT_ROOT, "outputs", "figures")
os.makedirs(figures_dir, exist_ok=True)

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

print("Project root:", PROJECT_ROOT)
print("Figures dir: ", figures_dir)

## 2. Carregamento dos Metadados

In [ ]:
metadata_dir = os.path.join(PROJECT_ROOT, "data", "metadata")

df       = pd.read_csv(os.path.join(metadata_dir, "train_metadata.csv"))
train_df = pd.read_csv(os.path.join(metadata_dir, "train_split.csv"))
val_df   = pd.read_csv(os.path.join(metadata_dir, "val_split.csv"))

print(f"Dataset completo: {len(df)} imagens")
print(f"Split treino:     {len(train_df)} imagens")
print(f"Split validação:  {len(val_df)} imagens")
print()
print(df["label"].value_counts().rename({0: "Normal", 1: "Pneumonia"}))

## 3. Distribuição de Classes — Motivação para o Class Weighting

**Achado que fundamenta a Hipótese 3 (H3).**

O desbalanceamento entre as classes impõe um viés natural ao modelo:
sem qualquer intervenção, o modelo tende a favorecer a classe majoritária
(Pneumonia), o que pode resultar em alto Recall mas baixa Specificity,
ou vice-versa dependendo da distribuição.

A ponderação de classes (class weighting) corrige esse viés atribuindo
maior peso à classe minoritária na função de perda.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 5))

COLORS_CLASS = {"Normal": "#5B8DB8", "Pneumonia": "#C0504D"}

# --- Plot 1: Dataset completo ---
counts_full = df["label"].value_counts().rename({0: "Normal", 1: "Pneumonia"})
bars = axes[0].bar(counts_full.index, counts_full.values,
                   color=[COLORS_CLASS[k] for k in counts_full.index],
                   edgecolor="white", width=0.5)
for bar, val in zip(bars, counts_full.values):
    pct = val / counts_full.sum() * 100
    axes[0].text(bar.get_x()+bar.get_width()/2, bar.get_height()+20,
                 f"{val}\n({pct:.1f}%)", ha="center", fontsize=10, fontweight="bold")
axes[0].set_title("Dataset Completo", fontsize=11, fontweight="bold")
axes[0].set_ylabel("Número de imagens", fontsize=10)
axes[0].set_ylim(0, max(counts_full.values) * 1.2)
axes[0].spines[["top","right"]].set_visible(False)

# --- Plot 2: Split Treino ---
counts_train = train_df["label"].value_counts().rename({0: "Normal", 1: "Pneumonia"})
bars2 = axes[1].bar(counts_train.index, counts_train.values,
                    color=[COLORS_CLASS[k] for k in counts_train.index],
                    edgecolor="white", width=0.5)
for bar, val in zip(bars2, counts_train.values):
    pct = val / counts_train.sum() * 100
    axes[1].text(bar.get_x()+bar.get_width()/2, bar.get_height()+10,
                 f"{val}\n({pct:.1f}%)", ha="center", fontsize=10, fontweight="bold")
axes[1].set_title("Split de Treino", fontsize=11, fontweight="bold")
axes[1].set_ylabel("Número de imagens", fontsize=10)
axes[1].set_ylim(0, max(counts_train.values) * 1.2)
axes[1].spines[["top","right"]].set_visible(False)

# --- Plot 3: Ratio de desbalanceamento por split ---
ratios = {
    "Completo": counts_full.get("Pneumonia", 0) / counts_full.get("Normal", 1),
    "Treino":   counts_train.get("Pneumonia", 0) / counts_train.get("Normal", 1),
    "Validação": (val_df["label"]==1).sum() / max((val_df["label"]==0).sum(), 1),
}
axes[2].bar(ratios.keys(), ratios.values(), color=["#7DBE7D","#5B8DB8","#E8A838"],
            edgecolor="white", width=0.4)
axes[2].axhline(1.0, color="gray", linestyle="--", linewidth=1.5,
                label="Balanceamento perfeito (ratio=1)")
for i, (k, v) in enumerate(ratios.items()):
    axes[2].text(i, v + 0.02, f"{v:.2f}x", ha="center",
                 fontsize=11, fontweight="bold")
axes[2].set_title("Ratio Pneumonia / Normal", fontsize=11, fontweight="bold")
axes[2].set_ylabel("Ratio", fontsize=10)
axes[2].legend(fontsize=9)
axes[2].spines[["top","right"]].set_visible(False)

fig.suptitle("Distribuição de Classes — Motivação para Class Weighting (H3)",
             fontsize=13, fontweight="bold", y=1.02)

# Caixa de decisão
fig.text(0.5, -0.05,
         "→ Desbalanceamento moderado (~2.7x) presente nos splits. "
         "Fundamenta o uso de class weighting (H3) na função de perda.",
         ha="center", fontsize=10, style="italic",
         bbox=dict(boxstyle="round", facecolor="#FFF3CD", alpha=0.8))

plt.tight_layout()
save_path = os.path.join(figures_dir, "eda_distribuicao_classes.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 4. Dimensões das Imagens — Motivação para Resize e Transfer Learning

**Achados que fundamentam:**
- Resize padronizado para 224×224 (padrão ImageNet)
- Transfer Learning: dataset de tamanho moderado não justifica treino do zero
- CenterCrop de 90% para reduzir artefatos periféricos

In [ ]:
print("Coletando dimensões das imagens (amostra de 300)...")

sample_df = df.sample(min(300, len(df)), random_state=42)
widths, heights, aspects = [], [], []

for path in sample_df["path"]:
    try:
        img = Image.open(path)
        w, h = img.size
        widths.append(w)
        heights.append(h)
        aspects.append(w / h)
    except Exception:
        pass

print(f"Largura — min: {min(widths)}, max: {max(widths)}, média: {np.mean(widths):.0f}")
print(f"Altura  — min: {min(heights)}, max: {max(heights)}, média: {np.mean(heights):.0f}")
print(f"Total de imagens no dataset: {len(df)}")
print()

# Categorias de tamanho
n_pequenas  = sum(1 for w, h in zip(widths, heights) if w < 500 or h < 500)
n_medias    = sum(1 for w, h in zip(widths, heights) if 500 <= w <= 1500 and 500 <= h <= 1500)
n_grandes   = sum(1 for w, h in zip(widths, heights) if w > 1500 or h > 1500)

print(f"Imagens pequenas (<500px):  {n_pequenas}")
print(f"Imagens médias (500-1500px): {n_medias}")
print(f"Imagens grandes (>1500px):  {n_grandes}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4.5))

# Largura
axes[0].hist(widths, bins=30, color="#5B8DB8", edgecolor="white")
axes[0].axvline(np.mean(widths), color="red", linestyle="--",
                linewidth=1.5, label=f"Média: {np.mean(widths):.0f}px")
axes[0].axvline(224, color="orange", linestyle=":",
                linewidth=2, label="Resize alvo: 224px")
axes[0].set_title("Distribuição de Largura", fontsize=11, fontweight="bold")
axes[0].set_xlabel("Pixels", fontsize=9)
axes[0].set_ylabel("Frequência", fontsize=9)
axes[0].legend(fontsize=8)
axes[0].spines[["top","right"]].set_visible(False)

# Altura
axes[1].hist(heights, bins=30, color="#C0504D", edgecolor="white")
axes[1].axvline(np.mean(heights), color="red", linestyle="--",
                linewidth=1.5, label=f"Média: {np.mean(heights):.0f}px")
axes[1].axvline(224, color="orange", linestyle=":",
                linewidth=2, label="Resize alvo: 224px")
axes[1].set_title("Distribuição de Altura", fontsize=11, fontweight="bold")
axes[1].set_xlabel("Pixels", fontsize=9)
axes[1].legend(fontsize=8)
axes[1].spines[["top","right"]].set_visible(False)

# Scatter Largura x Altura
axes[2].scatter(widths, heights, alpha=0.3, s=15, color="#7DBE7D")
axes[2].plot([0, max(max(widths), max(heights))],
             [0, max(max(widths), max(heights))],
             "k--", linewidth=1, label="Proporção 1:1")
axes[2].set_title("Largura × Altura", fontsize=11, fontweight="bold")
axes[2].set_xlabel("Largura (px)", fontsize=9)
axes[2].set_ylabel("Altura (px)", fontsize=9)
axes[2].legend(fontsize=8)
axes[2].spines[["top","right"]].set_visible(False)

fig.suptitle("Distribuição de Dimensões das Imagens — Motivação para Resize Padronizado",
             fontsize=13, fontweight="bold", y=1.02)

fig.text(0.5, -0.06,
         "→ Alta variabilidade de dimensões exige resize padronizado (224×224). "
         "Dataset de ~5k imagens favorece transfer learning sobre treino do zero.",
         ha="center", fontsize=10, style="italic",
         bbox=dict(boxstyle="round", facecolor="#FFF3CD", alpha=0.8))

plt.tight_layout()
save_path = os.path.join(figures_dir, "eda_dimensoes.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 5. Distribuição de Intensidade de Pixels por Classe

**Achados que fundamentam:**
- Diferenças de textura e densidade entre Normal e Pneumonia
- Normalização com média/std do ImageNet
- CNNs como escolha arquitetural (capturam padrões espaciais/texturais)

Imagens de pneumonia tendem a apresentar maior densidade
em regiões pulmonares (consolidações, infiltrados),
refletindo em distribuições de intensidade distintas.

In [ ]:
print("Calculando histogramas por classe (amostra de 150 por classe)...")

pixels_normal    = []
pixels_pneumonia = []
means_normal, means_pneumonia = [], []
stds_normal,  stds_pneumonia  = [], []

sample_normal    = df[df["label"] == 0].sample(min(150, (df["label"]==0).sum()), random_state=42)
sample_pneumonia = df[df["label"] == 1].sample(min(150, (df["label"]==1).sum()), random_state=42)

for path in sample_normal["path"]:
    try:
        arr = np.array(Image.open(path).convert("L").resize((224, 224))).flatten()
        pixels_normal.extend(arr)
        means_normal.append(arr.mean())
        stds_normal.append(arr.std())
    except Exception:
        pass

for path in sample_pneumonia["path"]:
    try:
        arr = np.array(Image.open(path).convert("L").resize((224, 224))).flatten()
        pixels_pneumonia.extend(arr)
        means_pneumonia.append(arr.mean())
        stds_pneumonia.append(arr.std())
    except Exception:
        pass

print(f"Normal    — média de intensidade: {np.mean(means_normal):.1f} ± {np.std(means_normal):.1f}")
print(f"Pneumonia — média de intensidade: {np.mean(means_pneumonia):.1f} ± {np.std(means_pneumonia):.1f}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Histograma de pixels
axes[0].hist(pixels_normal, bins=60, alpha=0.6, color="#5B8DB8",
             label="Normal", density=True)
axes[0].hist(pixels_pneumonia, bins=60, alpha=0.6, color="#C0504D",
             label="Pneumonia", density=True)
axes[0].set_title("Distribuição de Intensidade de Pixels\npor Classe",
                  fontsize=11, fontweight="bold")
axes[0].set_xlabel("Intensidade (0-255)", fontsize=9)
axes[0].set_ylabel("Densidade", fontsize=9)
axes[0].legend(fontsize=9)
axes[0].spines[["top","right"]].set_visible(False)

# Boxplot da média por imagem
data_means = [means_normal, means_pneumonia]
bp = axes[1].boxplot(data_means, labels=["Normal", "Pneumonia"],
                     patch_artist=True, notch=True,
                     medianprops=dict(color="black", linewidth=2))
for patch, color in zip(bp["boxes"], ["#5B8DB8", "#C0504D"]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
axes[1].set_title("Média de Intensidade por Imagem",
                  fontsize=11, fontweight="bold")
axes[1].set_ylabel("Intensidade média", fontsize=9)
axes[1].spines[["top","right"]].set_visible(False)

# Boxplot do desvio padrão
data_stds = [stds_normal, stds_pneumonia]
bp2 = axes[2].boxplot(data_stds, labels=["Normal", "Pneumonia"],
                      patch_artist=True, notch=True,
                      medianprops=dict(color="black", linewidth=2))
for patch, color in zip(bp2["boxes"], ["#5B8DB8", "#C0504D"]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
axes[2].set_title("Desvio Padrão de Intensidade por Imagem\n(proxy de heterogeneidade textural)",
                  fontsize=11, fontweight="bold")
axes[2].set_ylabel("Desvio padrão", fontsize=9)
axes[2].spines[["top","right"]].set_visible(False)

fig.suptitle("Análise de Intensidade de Pixels por Classe",
             fontsize=13, fontweight="bold", y=1.02)

fig.text(0.5, -0.06,
         "→ Diferenças de intensidade e textura entre classes confirmam que "
         "CNNs são adequadas: capturam padrões espaciais locais hierárquicos "
         "que separam as classes.",
         ha="center", fontsize=10, style="italic",
         bbox=dict(boxstyle="round", facecolor="#FFF3CD", alpha=0.8))

plt.tight_layout()
save_path = os.path.join(figures_dir, "eda_intensidade_pixels.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 6. Galeria Visual — Padrões Diagnósticos por Classe

Visualização comparativa de exemplos representativos de cada classe,
ilustrando os padrões visuais que os modelos devem aprender.

**O que observar:**
- **Normal**: campos pulmonares translúcidos, costelas bem definidas, sem opacidades
- **Pneumonia**: consolidações (regiões opacas/brancas), infiltrados, assimetrias

Esses padrões são espacialmente distribuídos e de textura variável,
justificando o uso de CNNs com filtros hierárquicos.

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(16, 7))

normal_samples    = df[df["label"] == 0].sample(5, random_state=42)
pneumonia_samples = df[df["label"] == 1].sample(5, random_state=42)

for i, (_, row) in enumerate(normal_samples.iterrows()):
    img = Image.open(row["path"]).convert("RGB")
    axes[0, i].imshow(img, cmap="gray")
    axes[0, i].set_title(f"Normal", fontsize=10, color="#5B8DB8", fontweight="bold")
    axes[0, i].axis("off")

for i, (_, row) in enumerate(pneumonia_samples.iterrows()):
    img = Image.open(row["path"]).convert("RGB")
    axes[1, i].imshow(img, cmap="gray")
    axes[1, i].set_title(f"Pneumonia", fontsize=10, color="#C0504D", fontweight="bold")
    axes[1, i].axis("off")

fig.suptitle("Galeria Visual — Exemplos Representativos por Classe",
             fontsize=13, fontweight="bold")

# Anotações de linha
fig.text(0.01, 0.73, "Normal", va="center", ha="left", rotation=90,
         fontsize=12, color="#5B8DB8", fontweight="bold")
fig.text(0.01, 0.27, "Pneumonia", va="center", ha="left", rotation=90,
         fontsize=12, color="#C0504D", fontweight="bold")

plt.tight_layout()
save_path = os.path.join(figures_dir, "eda_galeria_visual.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 7. Análise de Artefatos Periféricos — Motivação para CenterCrop

Imagens de raio-X clínicas frequentemente contêm:
- Bordas escuras (padding do detector)
- Marcações hospitalares (texto, logos)
- Dispositivos médicos (drenos, cabos de monitor)

O CenterCrop de 90% do resize remove sistematicamente
as bordas externas, reduzindo o risco de o modelo
aprender esses artefatos como features discriminativas.

In [ ]:
from torchvision import transforms

# Transformações para demonstração
resize_only = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

with_centercrop = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(int(224 * 0.9)),  # 201px
    transforms.ToTensor(),
])

sample_paths = df.sample(4, random_state=7)["path"].tolist()

fig, axes = plt.subplots(2, 4, figsize=(14, 7))

for i, path in enumerate(sample_paths):
    img_pil = Image.open(path).convert("RGB")

    # Resize apenas
    img_resize = transforms.Compose([transforms.Resize((224, 224))])(img_pil)
    axes[0, i].imshow(img_resize, cmap="gray")
    axes[0, i].set_title("Resize (224×224)", fontsize=9)
    axes[0, i].axis("off")

    # Com CenterCrop
    img_crop = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.CenterCrop(int(224 * 0.9))
    ])(img_pil)
    axes[1, i].imshow(img_crop, cmap="gray")
    axes[1, i].set_title("+ CenterCrop (90%)", fontsize=9, color="#7DBE7D")
    axes[1, i].axis("off")

    # Destacar bordas removidas
    for ax in [axes[0, i], axes[1, i]]:
        for spine in ax.spines.values():
            spine.set_visible(True)

fig.suptitle("Efeito do CenterCrop — Remoção de Bordas e Artefatos Periféricos",
             fontsize=13, fontweight="bold")

fig.text(0.5, -0.02,
         "→ CenterCrop de 90% remove regiões periféricas com artefatos "
         "(bordas escuras, marcações) sem perda de informação pulmonar central.",
         ha="center", fontsize=10, style="italic",
         bbox=dict(boxstyle="round", facecolor="#FFF3CD", alpha=0.8))

plt.tight_layout()
save_path = os.path.join(figures_dir, "eda_centercrop.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 8. Análise de Data Augmentation — Motivação para H2

Visualização comparativa entre o augmentation leve (Baseline)
e o augmentation forte (H2), aplicados sobre a mesma imagem.

**Hipótese 2:** augmentation mais intenso atuaria como regularização,
reduzindo sobreajuste e melhorando generalização.

In [ ]:
aug_light = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(int(224 * 0.9)),
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.RandomRotation(5),
])

aug_strong = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(int(224 * 0.9)),
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=0, scale=(0.9, 1.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
])

# Usar a mesma imagem para comparação
sample_path = df[df["label"] == 1].sample(1, random_state=10)["path"].values[0]
original    = Image.open(sample_path).convert("RGB")

fig, axes = plt.subplots(2, 5, figsize=(16, 7))

# Linha 0: Augmentation Leve
axes[0, 0].imshow(original, cmap="gray")
axes[0, 0].set_title("Original", fontsize=9, fontweight="bold")
axes[0, 0].axis("off")

for j in range(1, 5):
    aug_img = aug_light(original)
    axes[0, j].imshow(aug_img, cmap="gray")
    axes[0, j].set_title(f"Leve #{j}", fontsize=9, color="#5B8DB8")
    axes[0, j].axis("off")

# Linha 1: Augmentation Forte
axes[1, 0].imshow(original, cmap="gray")
axes[1, 0].set_title("Original", fontsize=9, fontweight="bold")
axes[1, 0].axis("off")

for j in range(1, 5):
    aug_img = aug_strong(original)
    axes[1, j].imshow(aug_img, cmap="gray")
    axes[1, j].set_title(f"Forte #{j}", fontsize=9, color="#C0504D")
    axes[1, j].axis("off")

fig.text(0.01, 0.73, "Aug. Leve", va="center", ha="left", rotation=90,
         fontsize=11, color="#5B8DB8", fontweight="bold")
fig.text(0.01, 0.27, "Aug. Forte", va="center", ha="left", rotation=90,
         fontsize=11, color="#C0504D", fontweight="bold")

fig.suptitle("Comparação Visual: Augmentation Leve vs Forte — Motivação para H2",
             fontsize=13, fontweight="bold")

fig.text(0.5, -0.02,
         "→ O augmentation forte altera mais agressivamente a geometria da imagem. "
         "Hipótese 2 testa se isso melhora a generalização sem distorcer padrões patológicos.",
         ha="center", fontsize=10, style="italic",
         bbox=dict(boxstyle="round", facecolor="#FFF3CD", alpha=0.8))

plt.tight_layout()
save_path = os.path.join(figures_dir, "eda_augmentation.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 9. Variabilidade Visual Intra-Classe — Motivação para Transfer Learning e H1

A variabilidade dentro de cada classe (diferentes pacientes,
idades, posicionamentos) impõe desafios adicionais ao modelo.

- **Alta variabilidade intra-classe** → Transfer Learning é essencial
  (representações genéricas do ImageNet aceleram convergência)
- **Padrões sutis em alguns casos** → Motivou testar DenseNet121 (H1),
  que propaga features de forma mais densa entre camadas

In [ ]:
# Calcular variância média dentro de cada classe
print("Calculando variância intra-classe...")

def get_image_vector(path, size=64):
    """Retorna imagem como vetor normalizado para comparação."""
    try:
        return np.array(Image.open(path).convert("L").resize((size, size))).flatten() / 255.0
    except Exception:
        return None

n_sample = 100
vecs_normal    = [v for path in df[df["label"]==0].sample(n_sample, random_state=42)["path"]
                  if (v := get_image_vector(path)) is not None]
vecs_pneumonia = [v for path in df[df["label"]==1].sample(n_sample, random_state=42)["path"]
                  if (v := get_image_vector(path)) is not None]

var_normal    = np.var(np.stack(vecs_normal),    axis=0).mean()
var_pneumonia = np.var(np.stack(vecs_pneumonia), axis=0).mean()

print(f"Variância intra-classe Normal:    {var_normal:.5f}")
print(f"Variância intra-classe Pneumonia: {var_pneumonia:.5f}")

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Mapa de variância pixel-a-pixel
var_map_normal    = np.var(np.stack(vecs_normal),    axis=0).reshape(64, 64)
var_map_pneumonia = np.var(np.stack(vecs_pneumonia), axis=0).reshape(64, 64)

im0 = axes[0].imshow(var_map_normal, cmap="hot", vmin=0)
axes[0].set_title(f"Variância Pixel-a-Pixel — Normal\n(média={var_normal:.4f})",
                  fontsize=11, fontweight="bold", color="#5B8DB8")
axes[0].axis("off")
plt.colorbar(im0, ax=axes[0], fraction=0.046)

im1 = axes[1].imshow(var_map_pneumonia, cmap="hot", vmin=0)
axes[1].set_title(f"Variância Pixel-a-Pixel — Pneumonia\n(média={var_pneumonia:.4f})",
                  fontsize=11, fontweight="bold", color="#C0504D")
axes[1].axis("off")
plt.colorbar(im1, ax=axes[1], fraction=0.046)

fig.suptitle("Variabilidade Visual Intra-Classe — Motivação para Transfer Learning",
             fontsize=13, fontweight="bold", y=1.02)

fig.text(0.5, -0.06,
         "→ Alta variabilidade intra-classe indica que o modelo precisa de "
         "representações robustas — justificando transfer learning em vez de treino do zero.",
         ha="center", fontsize=10, style="italic",
         bbox=dict(boxstyle="round", facecolor="#FFF3CD", alpha=0.8))

plt.tight_layout()
save_path = os.path.join(figures_dir, "eda_variabilidade_intraclasse.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 10. Síntese da EDA — Mapa de Decisões

Figura consolidada conectando cada achado à sua decisão metodológica.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
ax.axis("off")

# Dados da tabela
table_data = [
    ["Desbalanceamento de classes (~2.7x)",
     "Class Weighting na loss function",
     "H3 — Class Weight"],
    ["Dataset moderado (~5k imagens)",
     "Transfer Learning (ImageNet)",
     "Baseline, H1, H2, H3"],
    ["Alta variabilidade de dimensões",
     "Resize padronizado 224×224",
     "Todos os experimentos"],
    ["Artefatos nas bordas periféricas",
     "CenterCrop 90% após resize",
     "Todos os experimentos"],
    ["Padrões espaciais/texturais nas classes",
     "CNN como arquitetura base",
     "Todos os experimentos"],
    ["Variabilidade intra-classe alta",
     "Augmentation leve como regularização",
     "Baseline, H1, H3"],
    ["Hipótese: aug. forte melhora generalização",
     "Augmentation forte (rotação ±15°, jitter)",
     "H2 — Strong Aug"],
    ["Hipótese: arquitetura diferente capta\npadrões mais sutis",
     "DenseNet121 (dense connections)",
     "H1 — DenseNet121"],
]

col_labels = ["Achado do EDA", "Decisão Metodológica", "Experimento Relacionado"]
col_colors = [["#2C3E50"] * 3]

row_colors_map = [
    "#FADADD",  # H3 - destaque
    "#EAF2FB",
    "#EAF2FB",
    "#EAF2FB",
    "#EAF2FB",
    "#EAF2FB",
    "#FEF9E7",
    "#FEF9E7",
]

table = ax.table(
    cellText=table_data,
    colLabels=col_labels,
    loc="center",
    cellLoc="left"
)

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.2)

# Estilo do header
for col in range(3):
    table[0, col].set_facecolor("#2C3E50")
    table[0, col].set_text_props(color="white", fontweight="bold")

# Cores das linhas
for row_idx, color in enumerate(row_colors_map, start=1):
    for col in range(3):
        table[row_idx, col].set_facecolor(color)

# Negrito na linha do H3
for col in range(3):
    table[1, col].set_text_props(fontweight="bold")

ax.set_title("Síntese da EDA — Mapa de Achados e Decisões Metodológicas",
             fontsize=13, fontweight="bold", pad=20)

# Legenda
patches = [
    mpatches.Patch(color="#FADADD", label="Motivação principal para modelo recomendado (H3)"),
    mpatches.Patch(color="#FEF9E7", label="Hipóteses testadas (H1, H2)"),
    mpatches.Patch(color="#EAF2FB", label="Decisões de pré-processamento (todos)"),
]
ax.legend(handles=patches, loc="lower center",
          bbox_to_anchor=(0.5, -0.08), ncol=3, fontsize=9)

plt.tight_layout()
save_path = os.path.join(figures_dir, "eda_mapa_decisoes.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 11. Conclusão da EDA

A análise exploratória revelou quatro características fundamentais
do conjunto de dados que guiaram as escolhas metodológicas:

**1. Desbalanceamento de classes (~2.7x):** a predominância de casos
de Pneumonia impõe viés natural ao modelo. A ponderação de classes
(H3) é a intervenção mais direta para corrigir esse efeito,
aumentando a penalidade por erros na classe minoritária (Normal).

**2. Tamanho moderado do dataset (~5.2k imagens):** insuficiente
para treinar arquiteturas profundas do zero com estabilidade.
O transfer learning com pesos do ImageNet reduz o risco de
sobreajuste e acelera a convergência.

**3. Variabilidade de dimensões e artefatos periféricos:**
exige padronização cuidadosa via Resize + CenterCrop,
garantindo que o modelo processe regiões anatomicamente
relevantes e não bordas ou marcações hospitalares.

**4. Padrões espaciais e texturais discriminativos:**
consolidações, infiltrados e opacidades são padrões
locais e hierárquicos — exatamente o que filtros
convolucionais são projetados para capturar.

Esses achados fundamentam empiricamente todas as hipóteses
experimentais e as decisões de pré-processamento adotadas.


In [ ]:
print("Figuras da EDA geradas em:", figures_dir)
figuras = [
    "eda_distribuicao_classes.png",
    "eda_dimensoes.png",
    "eda_intensidade_pixels.png",
    "eda_galeria_visual.png",
    "eda_centercrop.png",
    "eda_augmentation.png",
    "eda_variabilidade_intraclasse.png",
    "eda_mapa_decisoes.png",
]
for f in figuras:
    print(" •", f)